In [21]:
# Block 1
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
import datetime
from google.colab import files
from keras.regularizers import l2
from keras.optimizers import Adam
import numpy as np

In [22]:
# Block 2
uploaded = files.upload()  # local upload

Saving neural_network_data.csv to neural_network_data (4).csv


In [23]:
# Block 3
raw_data = pd.read_csv("neural_network_data.csv")

# Remove any leading or trailing spaces in the column names
raw_data.columns = raw_data.columns.str.strip()

# Encode the 'Behavior' column labels as integers
label_encoder = LabelEncoder()
raw_data["Behavior"] = label_encoder.fit_transform(raw_data["Behavior"])

# Set new variable as just training data
training_inputs = raw_data.drop(columns=["Behavior"])

# Separate the 'Behavior' column as the output labels
outputs = raw_data["Behavior"]

# Convert the 'Behavior' column to one-hot encoding for multi-class classification
one_hot_behavior = to_categorical(raw_data['Behavior'])

# Split the data into training and testing sets
# 80% of the data is used for training and 20% for testing
input_train, input_test, output_train, output_test = train_test_split(training_inputs, one_hot_behavior, test_size=.20, random_state=420)

In [43]:
# Block 4
# Create model
# Define a neural network model using the Sequential API from Keras
model = Sequential(
    [
        Dense(15, input_shape=(input_train.shape[1],), activation='relu'),
        Dense(10, activation='relu', kernel_regularizer=l2(0.01)),
        Dense(one_hot_behavior.shape[1], activation='softmax')
    ]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
# Block 5
# Compile model
optimizer = Adam(learning_rate=0.005)  # Set learning rate Default lerning rate: 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
# Block 6
# Train model
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
model.fit(input_train, output_train, epochs=300, batch_size=25, validation_data=(input_test, output_test), callbacks=[early_stopping])

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2857 - loss: 1.4969 - val_accuracy: 0.2500 - val_loss: 1.5294
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.2857 - loss: 1.4670 - val_accuracy: 0.2500 - val_loss: 1.5176
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3571 - loss: 1.4428 - val_accuracy: 0.2500 - val_loss: 1.5101
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5000 - loss: 1.4281 - val_accuracy: 0.2500 - val_loss: 1.4995
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5000 - loss: 1.4145 - val_accuracy: 0.2500 - val_loss: 1.4894
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.5714 - loss: 1.4012 - val_accuracy: 0.2500 - val_loss: 1.4797
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5714 - loss: 1.3875 - val_accuracy: 0.2500 - val_loss: 1.4703
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 1.3735 - val_accuracy: 0.2500 - val_loss: 

In [46]:
# Block 7
# Save the model after training
model.save('my_behavior_model.h5')

In [47]:
# Block 8
# Load the model (when needed)
loaded_model = load_model('my_behavior_model.h5')

In [48]:
# Block 9
# Prepare a random scenario for prediction
random_scenario = np.random.rand(1, input_train.shape[1])  # Generate random values based on the number of features

# Print the generated random scenario
feature_names = training_inputs.columns  # Get feature names from the training data
scenario_dict = {feature_names[i]: random_scenario[0][i] for i in range(len(feature_names))}  # Create a dictionary for better readability
print("Random scenario for prediction:")
print(scenario_dict)

# Make predictions
predicted_output = loaded_model.predict(random_scenario)

Random scenario for prediction:
{'Health': 0.1961139919089756, 'Has-Knife': 0.12146293745082692, 'Has-Gun': 0.10128516728271031, 'Enemies': 0.9328162136061342}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [30]:
# Block 9 (Customize input for testing)
# Prepare a random scenario for prediction
# Define tweaking ranges for each feature
health_range = (5.0, 10.0)  # Health range: 5.0 to 10.0 (float)
knife_range = (0.0, 1.0)     # Has-Knife range: 0.0 to 1.0 (float)
gun_range = (0.0, 1.0)       # Has-Gun range: 0.0 to 1.0 (float)
enemies_range = (0.0, 3.0)   # Enemies range: 0.0 to 3.0 (float)

# Generate random values for each feature within the specified ranges
health_value = np.random.uniform(*health_range)  # Random float for Health
knife_value = np.random.uniform(*knife_range)    # Random float for Has-Knife
gun_value = np.random.uniform(*gun_range)        # Random float for Has-Gun
enemies_value = np.random.uniform(*enemies_range)  # Random float for Enemies

# Create the random scenario array
random_scenario = np.array([[health_value, knife_value, gun_value, enemies_value]])

# Print the generated random scenario
feature_names = training_inputs.columns  # Get feature names from the training data
scenario_dict = {feature_names[i]: random_scenario[0][i] for i in range(len(feature_names))}  # Create a dictionary for better readability
print("Random scenario for prediction:")
print(scenario_dict)

# Make predictions
predicted_output = loaded_model.predict(random_scenario)  # Reshape not necessary as it's already the correct shape



Random scenario for prediction:
{'Health': 8.340365872773244, 'Has-Knife': 0.49319847752385937, 'Has-Gun': 0.9723649345898212, 'Enemies': 1.7554609868539952}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [49]:
# Block 10
# Interpret Output
predicted_class = np.argmax(predicted_output, axis=1)
behavior_labels = label_encoder.inverse_transform(predicted_class)  # Convert to original labels
predicted_behavior = behavior_labels[0]
print(f"The predicted behavior is: {predicted_behavior}")

The predicted behavior is: Hide
